In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)\

import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from aif360.sklearn.datasets import fetch_compas

In [2]:
cor_thres = 0.5
cost_budget = 10
top_count = 20
c_inf = 5

In [3]:
urlfile= "https://raw.githubusercontent.com/samuel-yeom/fliptest/master/exact-ot/chicago-ssl-clean.csv"

X = pd.read_csv(urlfile)
sensitive_attribute = "SEX CODE CD"
df = clean_dataset(X,'SSL')
df.head()

,SSL SCORE,PREDICTOR RAT AGE AT LATEST ARREST,PREDICTOR RAT VICTIM SHOOTING INCIDENTS,PREDICTOR RAT VICTIM BATTERY OR ASSAULT,PREDICTOR RAT ARRESTS VIOLENT OFFENSES,PREDICTOR RAT GANG AFFILIATION,PREDICTOR RAT NARCOTIC ARRESTS,PREDICTOR RAT TREND IN CRIMINAL ACTIVITY,PREDICTOR RAT UUW ARRESTS,SEX CODE CD,RACE CODE CD
0,0,20-30,0,1,4,1,1,"(0.3, 7.3]",1,M,BLK
1,0,20-30,0,9,1,1,0,"(0.3, 7.3]",0,M,WHI
2,0,10-20,1,2,1,1,3,"(0.3, 7.3]",0,M,WHI
3,0,10-20,1,1,4,1,0,"(0.3, 7.3]",1,M,BLK
4,0,10-20,0,0,5,0,0,"(0.3, 7.3]",3,M,BLK


In [4]:
y = df['SSL SCORE']
X = df.drop('SSL SCORE', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features=[]

In [5]:
rules_with_atomic_correctness = load_rules_by_if("rulesSSL_atomic.data")
rules_with_cumulative_correctness = load_rules_by_if("rulesSSL_cumulative.data")

# Old Metrics

In [6]:
feature_weights = {'PREDICTOR RAT AGE AT LATEST ARREST':10,	'PREDICTOR RAT VICTIM SHOOTING INCIDENTS':1,	
'PREDICTOR RAT VICTIM BATTERY OR ASSAULT':1,	'PREDICTOR RAT ARRESTS VIOLENT OFFENSES':1,	'PREDICTOR RAT GANG AFFILIATION':1,
'PREDICTOR RAT NARCOTIC ARRESTS':1,	'PREDICTOR RAT TREND IN CRIMINAL ACTIVITY':1,	'PREDICTOR RAT UUW ARRESTS':1,	'SEX CODE CD':100,	'RACE CODE CD':100}

features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=True,
    #feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)

## Weighted Average

In [7]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric = "weighted-average",
    sort_strategy = "abs-diff-decr",
    top_count = 50,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0, PREDICTOR RAT NARCOTIC ARRESTS = 3, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PREDICTOR RAT VICTIM SHOOTING INCIDENTS = 0:
	Protected Subgroup 'BLK', 1.18% covered
		Make PREDICTOR RAT NARCOTIC ARRESTS = 0 with effectiveness 54.26%.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 45.74%.
		Make PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 1, PREDICTOR RAT NARCOTIC ARRESTS = 0 with effectiveness 0.00%.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 2 with effectiveness 43.62%.
		Aggregate cost of the above recourses = -6.13
	Protected Subgroup 'WHI', 1.14% covered
		Make PREDICTOR RAT NARCOTIC ARRESTS = 0 with effectiveness 44.00%.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 32.00%.
		Make PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 1, PREDICTOR RAT NARCOTIC ARRESTS = 0 with effectiveness 0.00%.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 2 with effectiveness 8.00%.
		Aggregate cost of the above recourses = -2.80
	Bias agains

## Minimum Cost Above Threshold -- Equal Cost of Effectiveness (Macro)

In [8]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",        
        # "remove-fair-rules",
        "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If PREDICTOR RAT AGE AT LATEST ARREST = 10-20, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 1, PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT UUW ARRESTS = 0:
	Protected Subgroup 'BLK', 12.56% covered
		Make PREDICTOR RAT AGE AT LATEST ARREST = 20-30, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectiveness 97.10%.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 30-40, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectiveness 99.90%.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 40-50, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectiveness 100.00%.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 50-60, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectiveness 100.00%.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 60-70, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectiveness 100.00%.
		Aggregate cost of the above recourses = 10.08
	Protected Subgroup 'WHI', 9.67% covered
		Make PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0 with effectivenes

## Number of Rules Above Threshold -- Equal Choice for Recourse

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.3, 7.3], SEX CODE CD = M:
	Protected Subgroup 'BLK', 7.04% covered
		No recourses for this subgroup!
		Aggregate cost of the above recourses = 0.00
	Protected Subgroup 'WHI', 12.31% covered
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 91.11%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 82.96%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1] with effectiveness 82.96%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 82.96%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3], SEX CODE CD = F with effectiveness 91.11%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1], SEX CODE CD = F with effectiveness 82.96%.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1], SEX CODE CD = F with effect

# New Metrics

In [10]:
# rules_with_cumulative = facts.cum_corr_costs_all(rules_by_if, X_test, model, sensitive_attribute="race", params=params)

In [11]:
# from facts.utils import load_rules_by_if, save_rules_by_if
# save_rules_by_if("rulesCompas_cumulative.data", rules_with_cumulative)

## Fairness of Mean Recourse Cost ($c_\infty$ implementation) -- Equal Mean Recourse

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-cinf",
    c_inf=c_inf,
    sort_strategy="generic-sorting",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1], PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, SEX CODE CD = M:
	Protected Subgroup 'BLK', 1.13% covered
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 0.00% and counterfactual cost = 0.03.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.3, 7.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.3, -0.2] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT NARCOTIC A

## Fairness of Mean Recourse Cost (conditional expectation implementation) -- Equal Conditional Mean Recourse

In [13]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If PREDICTOR RAT AGE AT LATEST ARREST = 10-20, PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1], PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0:
	Protected Subgroup 'BLK', 1.67% covered
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 20-30 with effectiveness 97.74% and counterfactual cost = 10.0.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 30-40 with effectiveness 100.00% and counterfactual cost = 10.0.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 40-50 with effectiveness 100.00% and counterfactual cost = 10.0.
		Make PREDIC

## Fairness of Recourse at Effectiveness Level -- Equal Cost of Effectiveness (Micro)

In [14]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-cheap-rules-above-thr-cor",
        # "keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If PREDICTOR RAT AGE AT LATEST ARREST = 10-20, PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0, PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 1:
	Protected Subgroup 'BLK', 5.68% covered
		Make PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0 with effectiveness 38.85% and counterfactual cost = 0.1.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 20-30, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0 with effectiveness 94.92% and counterfactual cost = 10.1.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 30-40, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0 with effectiveness 99.56% and counterfactual cost = 10.1.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 40-50, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0 with effectiveness 100.00% and counterfactual cost = 10.1.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 50-60, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0 with effectiveness 100.00% and counterfactual cost = 10.1.
		Make PREDICTOR RAT AGE AT LATEST ARREST = 60-70, PREDICTOR RAT VICTI

## Fairness of Effectiveness -- Equal Effectiveness

In [15]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True
)

If PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1], PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0:
	Protected Subgroup 'BLK', 1.68% covered
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 0.00% and counterfactual cost = 0.03.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.3, 7.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.3, -0.2] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1, PREDI

## Fairness of Recourse (using two-sample Kolmogorov-Smirnov test) -- Fair Effectiveness-Cost Trade-Off

In [16]:
# taken from the other notebooks
affected_pop_sizes = {"BLK": 8072, "WHI": 3271}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True
    # show_cumulative_plots=True
)

If PREDICTOR RAT ARRESTS VIOLENT OFFENSES = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.3, 7.3], PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0:
	Protected Subgroup 'BLK', 6.02% covered out of 8072
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 0.00% and counterfactual cost = 0.03.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 99.58% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 99.58% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1] with effectiveness 99.58% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.3, -0.2] with effectiveness 99.58% and counterfactual cost = 1.0.
		Make PREDICTOR RAT NAR

## Fairness of Effectiveness at Recourse Budget -- Equal Effectiveness within Budget (Micro)

In [17]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "remove-above-thr-cost"
        #"remove-below-thr",
        #"keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True
)

If PREDICTOR RAT GANG AFFILIATION = 0, PREDICTOR RAT NARCOTIC ARRESTS = 0, PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.2, -0.1], PREDICTOR RAT UUW ARRESTS = 0, PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0:
	Protected Subgroup 'BLK', 1.68% covered
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1 with effectiveness 0.00% and counterfactual cost = 0.03.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-8.200999999999999, -0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.1, 0.1] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.1, 0.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (0.3, 7.3] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT TREND IN CRIMINAL ACTIVITY = (-0.3, -0.2] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make PREDICTOR RAT NARCOTIC ARRESTS = 1, PREDI

## Get ranking of subgroups based on metrics

In [18]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [19]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])
budgets

[0.15229885057471265, 1.0, 10.0]

In [20]:
df = make_table(
    rules_with_both,
    sensitive_attribute_vals=["WHI", "BLK"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)

df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)

dropping on a non-lexsorted multi-index without a level parameter may impact performance.


In [21]:
df = df.loc[:, ~df.columns.duplicated()]
df.head()

subgroup  \
                                            subgroup   
0                      PREDICTOR RAT UUW ARRESTS = 0   
1                                    SEX CODE CD = M   
2         PREDICTOR RAT AGE AT LATEST ARREST = 10-20   
3        PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0   
4  PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, P...   

  (Equal Cost of Effectiveness(Macro), 0.3)        \
                                        WHI   BLK   
0                                       inf   inf   
1                                       inf   inf   
2                                      10.0  10.0   
3                                       inf   inf   
4                                       inf   inf   

  (Equal Cost of Effectiveness(Macro), 0.7)        \
                                        WHI   BLK   
0                                       inf   inf   
1                                       inf   inf   
2                                      10.0  10.0   
3                                       inf   inf   
4                                       inf   inf   

  (Equal Choice for Recourse, 0.3)     (Equal Choice for Recourse, 0.7)      \
                               WHI BLK                              WHI BLK   
0                                0   0                                0   0   
1                                0   0                                0   0   
2                               -5  -5                               -5  -5   
3                                0   0                                0   0   
4                                0   0                                0   0   

  Equal Effectiveness  ... (Equal Effectiveness within Budget, 10.0)  \
                  WHI  ...                                       WHI   
0            0.000000  ...                                  0.000000   
1            0.000000  ...                                       inf   
2            0.982002  ...                                  0.982002   
3            0.000000  ...                                  0.000000   
4            0.000000  ...                                  0.000000   

            (Equal Cost of Effectiveness(Micro), 0.3)        \
        BLK                                       WHI   BLK   
0  0.000000                                       inf   inf   
1       inf                                       inf   inf   
2  0.992935                                      10.0  10.0   
3  0.000000                                       inf   inf   
4  0.000000                                       inf   inf   

  (Equal Cost of Effectiveness(Micro), 0.7)        \
                                        WHI   BLK   
0                                       inf   inf   
1                                       inf   inf   
2                                      10.0  10.0   
3                                       inf   inf   
4                                       inf   inf   

  Equal(Conditional Mean Recourse)       Fair Effectiveness-Cost Trade-Off  \
                               WHI   BLK                             value   
0                              inf   inf                          0.000000   
1                              inf   inf                          0.000000   
2                             10.0  10.0                          0.059872   
3                              inf   inf                          0.000000   
4                              inf   inf                          0.000000   

        
  bias  
0  BLK  
1  BLK  
2  BLK  
3  BLK  
4  BLK  

[5 rows x 25 columns]

In [30]:
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_df
sensitive_attribute_vals=["WHI", "BLK"]
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=["WHI", "BLK"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

dropping on a non-lexsorted multi-index without a level parameter may impact performance.


In [31]:
rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals)
analysis_df = get_analysis_df(comb_df, sensitive_attribute_vals)

dropping on a non-lexsorted multi-index without a level parameter may impact performance.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [32]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                                        rank   
subgroup                                                                                       
PREDICTOR RAT UUW ARRESTS = 0                                                           Fair   
SEX CODE CD = M                                                                         Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20                                              Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0                                             Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...                                      Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
PREDICTOR RAT UUW ARRESTS = 0                        0.0         Fair   
SEX CODE CD = M                                      0.0         Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20           0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0          0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...   0.0         Fair   

                                                   (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                                        rank   
subgroup                                                                                       
PREDICTOR RAT UUW ARRESTS = 0                                                           Fair   
SEX CODE CD = M                                                                         Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20                                              Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0                                             Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...                                      Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
PREDICTOR RAT UUW ARRESTS = 0                        0.0         Fair   
SEX CODE CD = M                                      0.0         Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20           0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0          0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...   0.0         Fair   

                                                   (Equal Choice for Recourse, 0.3)  \
                                                                               rank   
subgroup                                                                              
PREDICTOR RAT UUW ARRESTS = 0                                                  Fair   
SEX CODE CD = M                                                                Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20                                     Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0                                    Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...                             Fair   

                                                                       \
                                                   score bias against   
subgroup                                                                
PREDICTOR RAT UUW ARRESTS = 0                        0.0         Fair   
SEX CODE CD = M                                      0.0         Fair   
PREDICTOR RAT AGE AT LATEST ARREST = 10-20           0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0          0.0         Fair   
PREDICTOR RAT VICTIM BATTERY OR ASSAULT = 0, PR...   0.0         Fair   

                                                   (Equal Cho

In [33]:
analysis_df

,Rank = 1 Count,WHI bias against Count,BLK bias against Count
"(Equal Cost of Effectiveness(Macro), 0.7)",627,1435,139
"(Equal Cost of Effectiveness(Micro), 0.7)",627,1435,139
"(Equal Cost of Effectiveness(Macro), 0.3)",371,1017,262
"(Equal Cost of Effectiveness(Micro), 0.3)",371,1017,262
Equal(Conditional Mean Recourse),116,3489,1361
"(Equal Choice for Recourse, 0.7)",16,2678,399
Fair Effectiveness-Cost Trade-Off,106,1072,5479
"(Equal Effectiveness within Budget, 0.15229885057471265)",40,1987,497
"(Equal Effectiveness within Budget, 1.0)",32,4090,963
Equal Effectiveness,4,695,3311
